In [22]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [9]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

In [10]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


### 프롬프트

In [20]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 사용자의 답변 , 사용자가 입력한 주관식 답변 그리고 노인들의 외로움과 한국형 외로움에 대한 연구를 바탕으로 어떤 한국형 외로움을 가지고있는지 설명해주세요. \\
    그 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. \\
    사용자의 이름은 {user.name}, 나이는 {user.age}, 성별은 {user.gender}, 학력은 {user.education}, 결혼 여부는 {user.merry}, 자녀 수는 {user.children}, 종교는 {user.religion}, 소득은 {user.income}, 경제 상태는 {user.economy_states}, 건강 상태는 {user.health_states}입니다. \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요
"""

모델 선언

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = SYS_PROMPT + '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-4o', temperature=0)

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
input_message =  """
Q:어떤 상황에서 외로움을 느끼시나요? \\
A:가장 외로웠던 상황은 가족들과 떨어져 살았을때 입니다. 내가 젊을때 기러기 아빠였어요. 밤에 혼자 집에가는데 집가도 아무도 없겠구나 싶어서 힘들더라구요 지금도 가끔 생각나요. \\
"""+INPUT_PROMPT
to_markdown(rag_chain.invoke("input_message"))


> 김영수님, 안녕하세요. 요즘 어떻게 지내고 계신가요? 가족들과의 관계에서 외로움을 느끼고 계신다고 들었어요. 특히, 가까운 사람들과의 대화에서 마음과 마음을 나누는 대화가 부족하다고 느끼신다고요. 
> 
> 김영수님께서 느끼시는 외로움은 '친밀해야 할 관계에서의 소원함'과 '애정, 공감, 존중을 받지 못해 속상함'으로 요약될 수 있을 것 같아요. 이는 가족들과의 대화가 정보 공유에 그치고, 진정한 마음의 소통이 부족하다고 느끼실 때 더욱 크게 다가오는 것 같습니다.
> 
> 김영수님께 적절한 대화 상대가 되기 위해, 저는 김영수님의 이야기를 경청하고, 공감하며, 존중하는 태도로 대화에 임하겠습니다. 김영수님께서 느끼시는 감정과 생각을 자유롭게 표현하실 수 있도록 도와드리고 싶어요. 
> 
> 혹시 최근에 특별히 마음에 두고 계신 일이 있으신가요? 또는 함께 나누고 싶은 이야기가 있으신가요? 언제든지 편하게 말씀해 주세요.